# Tennis Court Keypoints Training

## Download and extract dataset

Dataset found on [TennisCourtDetector ↗](https://github.com/yastrebksv/TennisCourtDetector) by yastrebksv

In [ ]:
# Download dataset from Google Drive

import urllib.request

url = 'https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040'
filename = 'tennis_court_det_dataset.zip'
urllib.request.urlretrieve(url, filename)

In [ ]:
# Unzip dataset

import zipfile

with zipfile.ZipFile("tennis_court_det_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("tennis_court_det_dataset")


## Start Code

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create Torch Dataset

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as file:
            self.data = json.load(file)
        
        # Transform images to standardize them
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    # Define length of dataset
    def __len__(self):
        return len(self.data)
    
    # Read items one by one and return the image and keypoints
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_train.json")
val_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

### Create model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [ ]:
model = model.to(device)

### Train model

You might need to adjust the `epochs` value depending on your available compute power and time requirements

Model was run locally (`Nvidia RTX 2060` GPU, `Intel Core i7-10750H` CPU, `32GB` of RAM), compute time was ~5h for 20 epochs.

Download the outputed model from `training/keypoints_model.pth` and add them to the `models` folder

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Can modify learning rate

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

In [ ]:
# Save predicted keypoints
torch.save(model.state_dict(), "keypoints_model.pth")